In [17]:
#====================================================================================================
# [bong][2022-03-07] 
# Elasticsearch test embedding 관련 코드를 sentence-bert를 이용하여 구현한 예제임(*미완성)
# => 실제 Elasticsearch 서버 접속이 안되서 테스트는 못해봄
# 참고 소스 : https://github.com/jtibshirani/text-embeddings
#            https://www.elastic.co/kr/blog/text-similarity-search-with-vectors-in-elasticsearch
#            https://velog.io/@jakeseo_me/문서-유사도-측정-구현하기-3-with-elasticsearch
#====================================================================================================
#elasticsearch 패키지 설치 
#!pip install elasticsearch

import json
import time

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import sys
sys.path.append("..")
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="es-test", logfilname="es-test")
device = GPU_info()
seed_everything(111)

logfilepath:es-test_2022-03-07.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [26]:
# sentence-bert embedding 모델 정의
from sentence_transformers import SentenceTransformer, util
model_path = '../model/bmc_fpt_kowiki20200920.train_model_0225-s-bert-nli-0303'
embedder = SentenceTransformer(model_path)
print(embedder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [2]:
##### INDEXING #####

def index_data():
    print("Creating the 'posts' index.")
    client.indices.delete(index=INDEX_NAME, ignore=[404])

    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
        client.indices.create(index=INDEX_NAME, body=source)

    docs = []
    count = 0

    # posts.json 파일을 일거와서 denser_vector 구함
    with open(DATA_FILE) as data_file:
        for line in data_file:
            line = line.strip()

            doc = json.loads(line)
            if doc["type"] != "question":
                continue

            docs.append(doc)
            count += 1

            if count % BATCH_SIZE == 0:
                index_batch(docs)  # index_batch를 호출
                docs = []
                print("Indexed {} documents.".format(count))

        if docs:
            index_batch(docs)# index_batch를 호출
            print("Indexed {} documents.".format(count))

    client.indices.refresh(index=INDEX_NAME)
    print("Done indexing.")

    # 실제 들어온 text에 대해 embedding 구함
def index_batch(docs):
    titles = [doc["title"] for doc in docs]
    title_vectors = embed_text(titles)  # embedding 구함

    requests = []
    for i, doc in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["title_vector"] = title_vectors[i]
        requests.append(request)
    bulk(client, requests) # elasticsearch 서버 로 값 넘겨줌??

In [3]:
##### SEARCHING #####

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return

def handle_query():
    
    # 입력 query text 들어옴 
    query = input("Enter query: ")

    embedding_start = time.time()
    # 입력 query text에 대한 embedding 수행후 vector 구함 
    query_vector = embed_text([query])[0]
    embedding_time = time.time() - embedding_start

    # cosinesimilarity query 구성함
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

    # Elasticsearch 서버로 쿼리 날러서 결과값 리턴
    search_start = time.time()
    response = client.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source": {"includes": ["title", "body"]}
        }
    )
    search_time = time.time() - search_start

    # 결과값 출력 
    print()
    print("{} total hits.".format(response["hits"]["total"]["value"]))
    print("embedding time: {:.2f} ms".format(embedding_time * 1000))
    print("search time: {:.2f} ms".format(search_time * 1000))
    for hit in response["hits"]["hits"]:
        print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
        print(hit["_source"])
        print()

In [27]:
##### EMBEDDING #####

def embed_text(text):
    #vectors = session.run(embeddings, feed_dict={text_ph: text})  #원래 코드
    
    # sentence-bert 를 이용하여 ,들어온 text:list들을 vercotr로 변환
    vectors = embedder.encode(text, convert_to_tensor=True)

    return [vector.tolist() for vector in vectors]

In [28]:
#### EMBEDDING 테스트 ####
corpus = ['오늘은 날씨가 흐리고 비가 온다',
          '식당에서 밥을 먹었다',
          '관광 버스를 타고 여행을 한다',
          '낚시를 해서 물고기를 많이 잡았다',
          '동물원에서 호랑이를 보았다',
          '선거일에는 투표하러 가야 한다',
          '마트에 가서 맛있는 배를 샀다',
          '도서관에서 시험 공부 하고 있다',
          '야구장에 가서 열심히 응원 했다']

denser_vector = embed_text(corpus)

print(type(denser_vector))
print(len(denser_vector))
print(denser_vector[1])

<class 'list'>
9
[-0.7663102149963379, -0.11338869482278824, -1.3734924793243408, 0.30791956186294556, 0.14291124045848846, 0.21879564225673676, 1.029555320739746, 0.2725464105606079, -0.6210060119628906, 0.05369127541780472, -1.790562629699707, 1.471078634262085, 0.05851005017757416, -0.4953974187374115, -0.3232819437980652, 0.40018993616104126, 0.16058295965194702, -0.44802749156951904, 0.8859685659408569, -0.23443087935447693, 0.02251826599240303, -0.14002272486686707, -0.32004180550575256, 0.1460217535495758, -1.0205867290496826, -0.9407118558883667, -2.113410472869873, -0.33415645360946655, 0.6583394408226013, 0.4283495545387268, 1.2802962064743042, 0.44680196046829224, -0.000547782052308321, -0.30099356174468994, 0.3932591378688812, -0.1578742116689682, -0.01356181874871254, 0.7133863568305969, -0.1832835078239441, 1.0212994813919067, -0.06186487153172493, 0.8582665324211121, -0.6443615555763245, 1.1187841892242432, -0.19590580463409424, -0.26291608810424805, -0.05302735418081283

In [7]:
##### Parameters #####

INDEX_NAME = "esdata"
INDEX_FILE = "esdata/index.json"

DATA_FILE = "esdata/posts.json"

BATCH_SIZE = 1000
SEARCH_SIZE = 5
GPU_LIMIT = 0.5


In [18]:
# elastricsearch 연결(*연결 실패남) E
#client = Elasticsearch("http://localhost:9200")
client = Elasticsearch()

ValueError: Either 'hosts' or 'cloud_id' must be specified

In [16]:
# 기존 data에 dense_vector구함
index_data()

# 쿼리에 대한 dense_vector구하고, 기존 data와 비교함
run_query_loop()

Creating the 'posts' index.


/tmp/ipykernel_49127/3923905961.py:5: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.delete(index=INDEX_NAME, ignore=[404])


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fec043d1850>: Failed to establish a new connection: [Errno 111] Connection refused))